In [ ]:

pip install requests bs4 beautifulsoup pandas

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_caic_weather_data(url):
    # Fetch the webpage content
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    all_data = []
    current_zone = None

    # Find all table elements
    tables = soup.find_all('table', class_='sortable')

    for table in tables:
        # Extract the zone name from the preceding h4 element
        zone_header = table.find_previous('h4')
        if zone_header:
            current_zone = zone_header.text.strip()

        # Extract column headers
        headers = [th.text.strip() for th in table.find_all('th')]

        # Extract rows
        rows = table.find_all('tr')[1:]  # Skip the header row

        # Process each row
        for row in rows:
            cols = row.find_all('td')
            if len(cols) >= len(headers):
                data = {headers[i]: cols[i].text.strip() for i in range(len(headers))}
                data['Zone'] = current_zone
                all_data.append(data)

    # Create a DataFrame
    df = pd.DataFrame(all_data)

    return df

# URL of the CAIC weather stations page
url = "https://classic.avalanche.state.co.us/caic/obs_stns/zones.php"

# Scrape the data
try:
    weather_data = scrape_caic_weather_data(url)
    
    # Save to CSV
    weather_data.to_csv("caic_weather_data.csv", index=False)
    print("Data has been scraped and saved to 'caic_weather_data.csv'")
    
    # Display first few rows
    print("\nFirst few rows of the scraped data:")
    print(weather_data.head())
    
    # Display summary information
    print("\nDataset summary:")
    print(weather_data.info())

except Exception as e:
    print(f"An error occurred while scraping the data: {str(e)}")

Data has been scraped and saved to 'caic_weather_data.csv'

First few rows of the scraped data:
               Station  Elev Temp MxTp MnTp DewP  RH Spd  Dir Gst Pcp1 Pcp24  \
0          Range South  6325   67   87   54    -   -   4  242   4    -     -   
1  US-40 at Elk Spring  6731   71   83   47   46  41   9  135  11    -     -   
2       U-40 Steamboat  6766   74   84   48   40  29   6  225  10    -     -   
3     Steamboat Sprngs  6877   67   83   50   48  51   3  330   -    -     -   
4               Gypsum  7340   68   87   52   50  52   4  173  11    -     -   

  PcpAc Sno24 SWE24 SnoHt SWE  Provider       Zone  
0     -     -     -     -   -  MesoWest  Steamboat  
1     -     -     -     -   -      CDOT  Steamboat  
2     -     -     -     -   -      CDOT  Steamboat  
3     -     -     -     -   -     METAR  Steamboat  
4     -     -     -     -   -      RAWS  Steamboat  

Dataset summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (t

In [6]:
# Step 1: Import necessary libraries
import pandas as pd

# Step 2: Load the data
file_path = '/Users/austinfinnell/Documents/GitHub/CAIC-Weather-Map/caic_weather_data.csv'  # Update this path to your local file
weather_data = pd.read_csv(file_path)

# Step 3: Filter the data for the 'Vail/Summit' zone
vail_summit_data = weather_data[weather_data['Zone'] == 'Vail/Summit']

# Step 4: Display the filtered data
print(vail_summit_data.head())  # Display the first few rows of the filtered dataset


                 Station    Elev Temp MxTp MnTp DewP  RH Spd  Dir Gst Pcp1  \
109     Mc Elroy/Kremmli  7412.0   64   81   41   50  59   7  110   -    -   
110  SH-9 Sum/Grand CO L  7797.0   62   82   42   46  54   6  180  10    -   
111   Elliot Ck Canal nr  7995.0   61   77   42   45  56   2   46   -    -   
112    Beaver Ck Village  8500.0   54   74   47    -   -   -    -   -    -   
113     CW9561 Kremmling  8626.0   59   74   45   52  76   1  350   -    -   

    Pcp24 PcpAc Sno24 SWE24 SnoHt   SWE   Provider         Zone  
109     -     -     -     -     -     -      METAR  Vail/Summit  
110     -     -     -     -     -     -       CDOT  Vail/Summit  
111     -     -     -     -     -     -   MesoWest  Vail/Summit  
112     -     -     -     -   0.0  0.00     SNOTEL  Vail/Summit  
113     -     -     -     -     -     -  APRSWXNET  Vail/Summit  


In [8]:
columns_to_drop = ['Pcp1', 'Pcp24', 'PcpAc', 'Sno24', 'SWE24', 'SnoHt', 'SWE', 'Gst']
vail_summit_data_cleaned = vail_summit_data.drop(columns=columns_to_drop)

# Step 5: Reset the index to remove the index numbers and drop the old index
vail_summit_data_cleaned.reset_index(drop=True, inplace=True)

# Step 6: Display the cleaned data
print(vail_summit_data_cleaned.head())  # Display the first few rows of the cleaned dataset

               Station    Elev Temp MxTp MnTp DewP  RH Spd  Dir   Provider  \
0     Mc Elroy/Kremmli  7412.0   64   81   41   50  59   7  110      METAR   
1  SH-9 Sum/Grand CO L  7797.0   62   82   42   46  54   6  180       CDOT   
2   Elliot Ck Canal nr  7995.0   61   77   42   45  56   2   46   MesoWest   
3    Beaver Ck Village  8500.0   54   74   47    -   -   -    -     SNOTEL   
4     CW9561 Kremmling  8626.0   59   74   45   52  76   1  350  APRSWXNET   

          Zone  
0  Vail/Summit  
1  Vail/Summit  
2  Vail/Summit  
3  Vail/Summit  
4  Vail/Summit  


In [9]:
stations = [
    {"name": "Mc Elroy/Kremmli", "provider": "METAR", "latitude": 40.05361, "longitude": -106.36889, "code": "K20V", "title": "Mc+Elroy/Kremmli+%28METAR%29+7412+ft", "elevation": 7412},
    {"name": "SH-9 Sum/Grand CO L", "provider": "CDOT", "latitude": 39.91342, "longitude": -106.32601, "code": "CO140", "title": "SH-9+Sum%2FGrand+CO+LN+%28CDOT%29+7797+ft", "elevation": 7797},
    {"name": "Elliot Ck Canal nr", "provider": "MesoWest", "latitude": 39.87361, "longitude": -106.33056, "code": "ELTC2", "title": "Elliot+Ck+Canal+nr+Green+Mountai+%28MesoWest%29+7995+ft", "elevation": 7995},
    {"name": "Beaver Ck Village", "provider": "SNOTEL", "latitude": 39.59917, "longitude": -106.51142, "code": "BCVC2", "title": "Beaver+Ck+Village+%28SNOTEL%29+8500+ft", "elevation": 8500},
    {"name": "CW9561 Kremmling", "provider": "APRSWXNET", "latitude": 40.1395, "longitude": -106.5312, "code": "C9561", "title": "CW9561+Kremmling+%28APRSWXNET%29+8626+ft", "elevation": 8626},
    {"name": "0.5 mi E of Vail -", "provider": "CDOT", "latitude": 39.62122, "longitude": -106.27850, "code": "CO160", "title": "0.5+mi+E+of+Vail+-+RTR+%28CDOT%29+8776+ft", "elevation": 8776},
    {"name": "Middle Fork Camp", "provider": "SNOTEL", "latitude": 39.79560, "longitude": -106.02730, "code": "MFKC2", "title": "Middle+Fork+Camp+%28SNOTEL%29+8940+ft", "elevation": 8940},
    {"name": "Dillon 1E", "provider": "MesoWest", "latitude": 39.62611, "longitude": -106.03556, "code": "DLLC2", "title": "Dillon+1E+%28MesoWest%29+9065+ft", "elevation": 9065},
    {"name": "Dowd Junction", "provider": "RAWS", "latitude": 39.62764, "longitude": -106.45217, "code": "DJTC2", "title": "Dowd+Junction+%28RAWS%29+9068+ft", "elevation": 9068},
    {"name": "0.9 mi W of CO-9 Fr", "provider": "CDOT", "latitude": 39.58391, "longitude": -106.10911, "code": "CO153", "title": "0.9+mi+W+of+CO-9+Frisco+%28CDOT%29+9134+ft", "elevation": 9134},
    {"name": "1.3 mi W of Frisco", "provider": "CDOT", "latitude": 39.56317, "longitude": -106.12958, "code": "CO155", "title": "1.3+mi+W+of+Frisco+%28CDOT%29+9256+ft", "elevation": 9256},
    {"name": "Keystone", "provider": "PWS", "latitude": 39.60992, "longitude": -105.95394, "code": "CAKEY", "title": "Keystone+%28PWS%29+9370+ft", "elevation": 9370},
    {"name": "2.4 mi E of Silvert", "provider": "CDOT", "latitude": 39.64654, "longitude": -106.02644, "code": "CO158", "title": "2.4+mi+E+of+Silverthorne+%28CDOT%29+9390+ft", "elevation": 9390},
    {"name": "Summit Ranch", "provider": "SNOTEL", "latitude": 39.71796, "longitude": -106.15802, "code": "SUMC2", "title": "Summit+Ranch+%28SNOTEL%29+9400+ft", "elevation": 9400},
    {"name": "Mccoy Park", "provider": "SNOTEL", "latitude": 39.60468, "longitude": -106.54128, "code": "MCYC2", "title": "Mccoy+Park+%28SNOTEL%29+9480+ft", "elevation": 9480},
    {"name": "0.6 mi E of CO-91 C", "provider": "CDOT", "latitude": 39.51496, "longitude": -106.14546, "code": "CO154", "title": "0.6+mi+E+of+CO-91+Copper+Mtn+%28CDOT%29+9668+ft", "elevation": 9668},
    {"name": "2.9 mi W of EJMT -", "provider": "CDOT", "latitude": 39.66206, "longitude": -105.98276, "code": "CO159", "title": "2.9+mi+W+of+EJMT+-+The+Box+%28CDOT%29+10208+ft", "elevation": 10208},
    {"name": "2.1 mi W of Vail Pa", "provider": "CDOT", "latitude": 39.56594, "longitude": -106.23739, "code": "CO172", "title": "2.1+mi+W+of+Vail+Pass+Summit+%28CDOT%29+10214+ft", "elevation": 10214},
    {"name": "Breckenridge 5S", "provider": "HADS", "latitude": 39.40861, "longitude": -106.04583, "code": "BKRC2", "title": "Breckenridge+5S+%28HADS%29+10230+ft", "elevation": 10230},
    {"name": "Vail Pass - CDOT Ya", "provider": "CAIC", "latitude": 39.57861, "longitude": -106.24722, "code": "CAVLP", "title": "Vail+Pass+-+CDOT+Yard+%28CAIC%29+10250+ft", "elevation": 10250},
    {"name": "Vail Mountain", "provider": "SNOTEL", "latitude": 39.61676, "longitude": -106.38006, "code": "VLMC2", "title": "Vail+Mountain+%28SNOTEL%29+10300+ft", "elevation": 10300},
    {"name": "Vail SA - Mid-Mtn", "provider": "VailResort", "latitude": 39.61751, "longitude": -106.38004, "code": "CAVMM", "title": "Vail+SA+-+Mid-Mtn+%28VailResort%29+10303+ft", "elevation": 10303},
    {"name": "Elliot Ridge", "provider": "SNOTEL", "latitude": 39.86400, "longitude": -106.42460, "code": "ELRC2", "title": "Elliot+Ridge+%28SNOTEL%29+10520+ft", "elevation": 10520},
    {"name": "Copper Mountain", "provider": "SNOTEL", "latitude": 39.48954, "longitude": -106.17095, "code": "CPMC2", "title": "Copper+Mountain+%28SNOTEL%29+10550+ft", "elevation": 10550},
    {"name": "Vail Pass", "provider": "CDOT", "latitude": 39.52958, "longitude": -106.21687, "code": "CO070", "title": "Vail+Pass+%28CDOT%29+10582+ft", "elevation": 10582},
    {"name": "1.7 mi W of EJMT -", "provider": "CDOT", "latitude": 39.67405, "longitude": -105.96589, "code": "CO157", "title": "1.7+mi+W+of+EJMT+-+Upper+RTR+%28CDOT%29+10610+ft", "elevation": 10610},
    {"name": "Vail SA - China Bow", "provider": "VailResort", "latitude": 39.59772, "longitude": -106.33835, "code": "CAVCB", "title": "Vail+SA+-+China+Bowl+%28VailResort%29+11031+ft", "elevation": 11031},
    {"name": "Grizzly Peak", "provider": "SNOTEL", "latitude": 39.64631, "longitude": -105.86973, "code": "GZPC2", "title": "Grizzly+Peak+%28SNOTEL%29+11100+ft", "elevation": 11100},
    {"name": "Vail SA - Blue Sky", "provider": "VailResort", "latitude": 39.57106, "longitude": -106.33239, "code": "CAVBS", "title": "Vail+SA+-+Blue+Sky+%28VailResort%29+11109+ft", "elevation": 11109},
    {"name": "EJT West Portal", "provider": "CDOT", "latitude": 39.67626, "longitude": -105.94356, "code": "CO126", "title": "EJT+West+Portal+%28CDOT%29+11126+ft", "elevation": 11126},
    {"name": "Ptarmigan", "provider": "USGS", "latitude": 39.49841944, "longitude": -106.2734778, "code": "USPTA", "title": "Ptarmigan+%28USGS%29+11202+ft", "elevation": 11202},
    {"name": "Vail SA - PHQ", "provider": "VailResort", "latitude": 39.60548, "longitude": -106.35657, "code": "CAVPQ", "title": "Vail+SA+-+PHQ+%28VailResort%29+11248+ft", "elevation": 11248},
    {"name": "Boss Basin", "provider": "USGS", "latitude": 39.47235278, "longitude": -106.2628, "code": "USBBN", "title": "Boss+Basin+%28USGS%29+11259+ft", "elevation": 11259},
    {"name": "Fremont Pass", "provider": "CDOT", "latitude": 39.36726, "longitude": -106.18777, "code": "CO121", "title": "Fremont+Pass+%28CDOT%29+11318+ft", "elevation": 11318},
    {"name": "Hoosier Pass", "provider": "SNOTEL", "latitude": 39.36055, "longitude": -106.05994, "code": "HOOC2", "title": "Hoosier+Pass+%28SNOTEL%29+11400+ft", "elevation": 11400},
    {"name": "Fremont Pass", "provider": "SNOTEL", "latitude": 39.37991, "longitude": -106.19681, "code": "FMTC2", "title": "Fremont+Pass+%28SNOTEL%29+11400+ft", "elevation": 11400},
    {"name": "A-Basin SA-MidMtn", "provider": "A-BasinSA", "latitude": 39.63227, "longitude": -105.86910, "code": "CAABM", "title": "A-Basin+SA-MidMtn+%28A-BasinSA%29+11660+ft", "elevation": 11660},
    {"name": "A-Basin SA-Pali", "provider": "A-BasinSA", "latitude": 39.63334, "longitude": -105.87870, "code": "CAABP", "title": "A-Basin+SA-Pali+%28A-BasinSA%29+11920+ft", "elevation": 11920},
    {"name": "Copper Mountain", "provider": "METAR", "latitude": 39.47523, "longitude": -106.15228, "code": "KCCU", "title": "Copper+Mountain+%28METAR%29+12074+ft", "elevation": 12074},
    {"name": "A-Basin SA-Summit", "provider": "A-BasinSA", "latitude": 39.62455, "longitude": -105.87630, "code": "CAABT", "title": "A-Basin+SA-Summit+%28A-BasinSA%29+12462+ft", "elevation": 12462}
]

In [11]:
stations_df = pd.DataFrame(stations)

# Step 7: Merge the weather data with stations DataFrame
merged_data = vail_summit_data_cleaned.merge(
    stations_df[['name', 'latitude', 'longitude', 'code']],
    left_on='Station',
    right_on='name',
    how='left'
)

# Step 8: Drop the redundant 'name' column after merging
merged_data.drop(columns=['name'], inplace=True)

# Step 9: Display the merged data with lat and long
print(merged_data.head())  # Display the first few rows of the merged dataset

               Station    Elev Temp MxTp MnTp DewP  RH Spd  Dir   Provider  \
0     Mc Elroy/Kremmli  7412.0   64   81   41   50  59   7  110      METAR   
1  SH-9 Sum/Grand CO L  7797.0   62   82   42   46  54   6  180       CDOT   
2   Elliot Ck Canal nr  7995.0   61   77   42   45  56   2   46   MesoWest   
3    Beaver Ck Village  8500.0   54   74   47    -   -   -    -     SNOTEL   
4     CW9561 Kremmling  8626.0   59   74   45   52  76   1  350  APRSWXNET   

          Zone  latitude  longitude   code  
0  Vail/Summit  40.05361 -106.36889   K20V  
1  Vail/Summit  39.91342 -106.32601  CO140  
2  Vail/Summit  39.87361 -106.33056  ELTC2  
3  Vail/Summit  39.59917 -106.51142  BCVC2  
4  Vail/Summit  40.13950 -106.53120  C9561  


In [12]:
import pandas as pd
import numpy as np
from metpy.calc import wind_components
from metpy.units import units
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from metpy.plots import StationPlot, StationPlotLayout

In [16]:
import ssl
import certifi
import urllib.request

# Use certifi to create an SSL context
context = ssl.create_default_context(cafile=certifi.where())

# Use the SSL context when opening a URL
response = urllib.request.urlopen('https://example.com', context=context)
print(response.read())


b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    

In [18]:


# Step 2: Convert necessary columns to appropriate data types
merged_data['temperature'] = merged_data['Temp'].replace('-', np.nan).astype(float)
merged_data['dewpoint'] = merged_data['DewP'].replace('-', np.nan).astype(float)
merged_data['windDir'] = merged_data['Dir'].replace('-', np.nan).astype(float)
merged_data['windSpeed'] = merged_data['Spd'].replace('-', np.nan).astype(float)

# Step 3: Prepare data for plotting
data = dict()
data['stid'] = np.array(merged_data['code'])
data['latitude'] = np.array(merged_data['latitude'])
data['longitude'] = np.array(merged_data['longitude'])
data['air_temperature'] = np.array(merged_data['temperature'], dtype=float) * units.degC
data['dew_point_temperature'] = np.array(merged_data['dewpoint'], dtype=float) * units.degC

# Handling wind components
direction = np.array(merged_data['windDir'].fillna(np.nan))
direction = np.where(direction == -9999.0, np.nan, direction)  # Replace missing or erroneous data

u, v = wind_components(np.array(merged_data['windSpeed'].fillna(0)) * units('knots'), direction * units.degree)
data['eastward_wind'], data['northward_wind'] = u, v

# Step 4: Set up the plot
proj = ccrs.LambertConformal(central_longitude=-105, central_latitude=39, standard_parallels=[39])
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1, projection=proj)

# Add various map elements
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.LAKES)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.STATES)
ax.add_feature(cfeature.BORDERS, linewidth=2)

# Set plot bounds for central Colorado
ax.set_extent([-107, -104, 38, 41], crs=ccrs.PlateCarree())  # Adjust this extent for your specific region

# Winds, temps, dewpoint, station id
custom_layout = StationPlotLayout()
custom_layout.add_barb('eastward_wind', 'northward_wind', units='knots')
custom_layout.add_value('NW', 'air_temperature', fmt='.0f', units='degC', color='darkred')
custom_layout.add_value('SW', 'dew_point_temperature', fmt='.0f', units='degC', color='darkgreen')

stationplot = StationPlot(ax, data['longitude'], data['latitude'], clip_on=True, transform=ccrs.PlateCarree(), fontsize=10)
stationplot.plot_text((2, 0), data['stid'])
custom_layout.plot(stationplot, data)

plt.title("Weather Data for Central Colorado")
plt.show()


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

<Figure size 1000x700 with 1 Axes>